![](https://import.cdn.thinkific.com/220744/BExaQBPPQairRWFqxFbK_logo_mastermind_web_png)

¡Ahora es **vuestro turno!**

En este proyecto final os he adjuntado un Dataset con 27.075 juegos en **Steam**, la popular plataforma de juegos de PC. Esta lista está actualizada desde el inicio hasta Abril de 2019.

Ya que tenéis los conocimientos, os pediré que me contestéis a **10 preguntas**:
1. Cual es el TOP10 de juegos más jugados?
2. Cuál es el género de videojuegos más vendido?
3. Qué géneros tienen mejores reviews de media?
4. Cuánto ha ganado el desarrollador que más juegos ha vendido?
5. Cómo afecta a las reviews que un juego sea Early Access?
6. Cuál es el desarrollador (o desarrolladores) que más tipos de juego y cantidad ha desarrollado?
7. En qué mes, desde que tenemos datos, se han publicado más juegos?
8. Cuántas horas se juega de media a cada tipo de juego mensualmente?
9. Los juegos Free to Play tienen mejores o peores reviews?
10. Cómo ha evolucionado el número de achievements (trofeos) por juego?

Os dejamos via libre para contestar a estas preguntas como más os apetezca. Recordad que no hay nada de malo en consultar por internet si tenéis dudas. Las documentaciones oficiales o [Stack Overflow](https://stackoverflow.com/) son sitios geniales para ello!

Al acabar este proyecto, publicadlo en el foro donde está colgado para ver quién es el mejor!

Os dejo la guía de pasos a seguir.

Muchos ánimos!!




# Importando librerías

In [35]:
import urllib.request as Req
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Consiguiendo los datos

### Conseguir link

In [36]:
url = 'https://neo-mastermind.s3.amazonaws.com/uploads/froala_editor/files/SteamDataset-220424-201858.zip'

### Descargar el archivo

In [37]:
Req.urlretrieve(url, 'steam.zip')

('steam.zip', <http.client.HTTPMessage at 0x2325ee69340>)

### Descomprimir el archivo

In [38]:
steam_zip = ZipFile('steam.zip')

In [39]:
steam_zip.filelist

[<ZipInfo filename='steam.csv' compress_type=deflate external_attr=0x20 file_size=3425223 compress_size=979195>]

In [40]:
archivo = steam_zip.open('steam.csv')

### Construir DataFrame

In [41]:
df = pd.read_csv(archivo, index_col=0)

In [42]:
df

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,free_to_play,early_access
0,Counter-Strike,2000-11-01,Valve,0,124534,3339,17612,7.19€,15000000.0,True,False,False,False,False,False,False,False
1,Team Fortress Classic,1999-04-01,Valve,0,3318,633,277,3.99€,7500000.0,True,False,False,False,False,False,False,False
2,Day of Defeat,2003-05-01,Valve,0,3416,398,187,3.99€,7500000.0,True,False,False,False,False,False,False,False
3,Deathmatch Classic,2001-06-01,Valve,0,1273,267,258,3.99€,7500000.0,True,False,False,False,False,False,False,False
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,0,5250,288,624,3.99€,7500000.0,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27098,Google Spotlight Stories: Piggy,2018-06-06,Google Inc.,0,15,0,0,0.0€,10000.0,False,True,False,False,False,False,True,False
27099,Sausage Sports Club,2018-07-19,Chris Wade,18,16,3,0,10.99€,10000.0,True,True,False,False,False,False,False,False
27100,Party Panic,2017-08-07,Everglow Interactive Inc.,28,619,226,84,12.99€,75000.0,True,False,False,False,False,False,False,False
27101,SoulHunt,2016-12-23,WinterCell Studios,18,56,19,0,5.99€,10000.0,True,False,False,False,False,False,False,True


# Limpieza y manipulación de datos

## Quitar datos duplicados o irrelevantes

## Arreglar errores estructurales

### Definir errores

Recuerda hacer una exploración de datos para encontrar estos errores. Os confirmo que hay unos cuantos!

## Filtrar outliers 

## Lidiar con NANs

## Validar nuestros datos

# EDA

## Cual es el TOP10 de juegos más jugados?

## Cuál es el género de videojuegos más vendido?

## Qué géneros tienen mejores reviews de media?

## Cuánto ha ganado el desarrollador que más juegos ha vendido?

## Cómo afecta a las reviews que un juego sea Early Access?

## Cuál es el desarrollador (o desarrolladores) que más tipos de juego y cantidad ha desarrollado?

## En qué mes, desde que tenemos datos, se han publicado más juegos?

## Cuántas horas se juega de media a cada tipo de juego mensualmente?

## Los juegos Free to Play tienen mejores o peores reviews?

## Cómo ha evolucionado el número de achievements (trofeos) por juego?

## Bonus! Si tienes curiosidad, y quieres contestar más preguntas, puedes resolverlas aquí mismo!